In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!pip install fair-esm
!pip install biopython

In [ ]:
import torch
import esm
import pandas as pd

# Load ESM-2 model  esm1v_t33_650M_UR90S_1()  esm2_t6_8M_UR50D,esm2_t48_15B_UR50D,esm2_t36_3B_UR50D,esm2_t30_150M_UR50D,esm2_t12_35M_UR50D
#https://github.com/facebookresearch/esm
model, alphabet = esm.pretrained.esm2_t12_35M_UR50D() #esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Read protein sequences from a FASTA file
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence:
                    sequences.append(sequence)
                    sequence = ''
            else:
                sequence += line.strip()
        if sequence:
            sequences.append(sequence)
    return sequences

path = '/content/drive/MyDrive/Watashara_Projects/neurotoxic/'
sequences = read_fasta(path+'Features_extraction/datasets/peptides/cross_val_dataset_peptides.fasta')

# Reduce batch size (e.g., to 2)
batch_size = 1
batches = [sequences[i:i+batch_size] for i in range(0, len(sequences), batch_size)]

# Initialize list to accumulate representations
all_sequence_representations = []

# Process batches
for batch_idx, batch in enumerate(batches):
    data = [(f'protein{i}', seq) for i, seq in enumerate(batch)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    # Accumulate representations
    all_sequence_representations.extend(sequence_representations)

# Convert tensor representations to numpy arrays
all_sequence_representations_numpy = [tensor.numpy() for tensor in all_sequence_representations]

# Convert numpy arrays to list of lists
all_sequence_representations_list = [list(arr) for arr in all_sequence_representations_numpy]

# Write to CSV
feat = pd.DataFrame(all_sequence_representations_list)
feat.to_csv(path+'features/peptides/TRN_esm2_640_Train.csv', index=False, header=False)


In [ ]:
#cross_val_dataset_protein independent_dataset_protein
#cross_val_dataset_combined independent_dataset_combined

In [ ]:
import torch
import esm
import pandas as pd

# Load ESM-2 model  esm1v_t33_650M_UR90S_1()  esm2_t6_8M_UR50D,esm2_t48_15B_UR50D,esm2_t36_3B_UR50D,esm2_t30_150M_UR50D,esm2_t12_35M_UR50D
#https://github.com/facebookresearch/esm
model, alphabet = esm.pretrained.#esm2_t6_8M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results

# Read protein sequences from a FASTA file
def read_fasta(file_path):
    sequences = []
    with open(file_path, 'r') as file:
        sequence = ''
        for line in file:
            if line.startswith('>'):
                if sequence:
                    sequences.append(sequence)
                    sequence = ''
            else:
                sequence += line.strip()
        if sequence:
            sequences.append(sequence)
    return sequences

path = '/content/drive/MyDrive/Watashara_Projects/neurotoxic/'
sequences = read_fasta(path+'Features_extraction/datasets/peptides/cross_val_dataset_peptides.fasta')

# Reduce batch size (e.g., to 2)
batch_size = 1
batches = [sequences[i:i+batch_size] for i in range(0, len(sequences), batch_size)]

# Initialize list to accumulate representations
all_sequence_representations = []

# Process batches
for batch_idx, batch in enumerate(batches):
    data = [(f'protein{i}', seq) for i, seq in enumerate(batch)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)
    batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[6], return_contacts=True)
    token_representations = results["representations"][6]

    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, tokens_len in enumerate(batch_lens):
        sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))

    # Accumulate representations
    all_sequence_representations.extend(sequence_representations)

# Convert tensor representations to numpy arrays
all_sequence_representations_numpy = [tensor.numpy() for tensor in all_sequence_representations]

# Convert numpy arrays to list of lists
all_sequence_representations_list = [list(arr) for arr in all_sequence_representations_numpy]

# Write to CSV
feat = pd.DataFrame(all_sequence_representations_list)
feat.to_csv(path+'features/peptides/TRN_esm2_320_Train.csv', index=False, header=False)


KeyboardInterrupt: 